In [1]:
# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# Specify the directory path where `assignemnt1.ipynb` exists.
# For example, if you saved `assignment1.ipynb` in `/gdrive/My Drive/cs470/assignment1` directory,
# then set root = '/gdrive/My Drive/cs470/assignment1'
root = '/gdrive/My Drive/colab'

In [3]:
from PIL import Image
from tqdm import tqdm
from pathlib import Path
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import requests
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

import pickle, joblib

#GO TO 'Prepare for Learning' CELL AFTER RUN THIS CELL

In [35]:
#Definition of functions

winrate_list = {266 : ("Aatrox", 46.07), 103 : ("Ahri", 50.59), 84 : ("Akali", 48.85),12 : ("Alistar", 50.27), 32 : ("Amumu", 53.24), 34 : ("Anivia", 54.33), 1 : ("Annie", 47.58), 523 : ("Aphelios", 53.4), 22 : ("Ashe", 49.57), 136 : ("AurelionSol", 44.32), 268 : ("Azir", 54.58), 432 : ("Bard", 54.09), 53 : ("Blitzcrank", 46.99), 63 : ("Brand", 50.24), 201 : ("Braum", 56.19), 51 : ("Caitlyn", 48.48), 164 : ("Camile", 50.58), 69 : ("Cassiopeia", 52.37), 31 : ("Chogath", 50.17), 42 : ("Corki", 50.89), 122 : ("Darius", 54.97), 131 : ("Diana", 49.24), 119 : ("Draven", 51.61), 36 : ("DrMundo", 51.46), 245 : ("Ekko", 50.67), 60 : ("Elise", 49.19), 28 : ("Evelynn", 49.54), 81 : ("Ezreal", 47.24), 9 : ("Fiddlesticks", 52.9), 114 : ("Fiora", 50.27), 105 : ("Fizz", 51.33), 3 : ("Galio", 47.29), 41 : ("Gangplank", 45.86), 86 : ("Garen", 47.19), 150 : ("Gnar", 55.04), 79 : ("Gragas", 43.72), 104 : ("Graves", 49.45), 120 : ("Hecarim", 47.45), 74 : ("Heimerdinger", 51.27), 420 : ("Illaoi", 52.8), 39 : ("Irelia", 42.59), 427 : ("Ivern", 44.7), 40 : ("Janna", 47.79), 59 : ("JarvanIV", 46.99), 24 : ("Jax", 45.92), 126 : ("Jayce", 52.2), 202 : ("Jhin", 51.11), 222 : ("Jinx", 48.95), 145 : ("Kaisa", 50), 429 : ("Kalista", 46.35), 43 : ("Karma", 47.47), 30 : ("Karthus", 48.71), 38 : ("Kassadin", 54.94), 55 : ("Katarina", 49.53), 10 : ("Kayle", 57.83), 141 : ("Kayn", 52.49), 85 : ("Kennen", 52.4), 121 : ("Khazix", 50.16), 203 : ("Kindred", 51.16), 240 : ("Kled", 51.58), 96 : ("KogMaw", 42.54), 7 : ("Leblanc", 49.97), 64 : ("LeeSin", 47.77), 89 : ("Leona", 50.75), 876 : ("Lillia", 47.75), 127 : ("Lissandra", 42.01), 236 : ("Lucian", 48.19), 117 : ("Lulu", 51.81), 99 : ("Lux", 48.26), 54 : ("Malphite", 50.72), 90 : ("Malzahar", 53.56), 57 : ("Maokai", 53.79), 11 : ("MasterYi", 48.12), 21 : ("MissFortune", 53.23), 62 : ("MonkeyKing", 49.26), 82 : ("Mordekaiser", 51.24), 25 : ("Morgana", 48.29), 267 : ("Nami", 48.42), 75 : ("Nasus", 48.66), 111 : ("Nautilus", 47.63), 518 : ("Neeko", 50.85), 76 : ("Nidalee", 48.05), 56 : ("Nocturne", 50.76), 20 : ("Nunu", 51.48), 2 : ("Olaf", 47.78), 61 : ("Orianna", 48.73), 516 : ("Ornn", 55.78), 80 : ("Pantheon", 47.5), 78 : ("Poppy", 47.64), 555 : ("Pyke", 46.6), 246 : ("Qiyana", 48.38), 133 : ("Quinn", 49.94), 497 : ("Rakan", 46.97), 33 : ("Rammus", 54.14), 421 : ("RekSai", 46.06), 58 : ("Renekton", 45.85), 107 : ("Rengar", 49.17), 92 : ("Riven", 51.62), 68 : ("Rumble", 50.2), 13 : ("Ryze", 47.67), 360 : ("Samira", 48.92), 113 : ("Sejuani", 50.69), 235 : ("Senna", 48.49), 147 : ("Seraphine", 51.25), 875 : ("Sett", 47.67), 35 : ("Shaco", 50.35), 98 : ("Shen", 51.07), 102 : ("Shyvana", 50.75), 27 : ("Singed", 51.35), 14 : ("Sion", 48.5), 15 : ("Sivir", 45.58), 72 : ("Skarner", 53.79), 37 : ("Sona", 56.07), 16 : ("Soraka", 52.05), 50 : ("Swain", 50.06), 517 : ("Sylas", 49.95), 134 : ("Syndra", 50.26), 223 : ("TahmKench", 58.37), 163 : ("Taliyah", 60.31), 91 : ("Talon", 49.37), 44 : ("Taric", 51.54), 17 : ("Teemo", 55.34), 412 : ("Thresh", 50.32), 18 : ("Tristana", 49.44), 48 : ("Trundle", 48.66), 23 : ("Tryndamere", 49.23), 4 : ("TwistedFate", 51.53), 29 : ("Twitch", 47.51), 77 : ("Udyr", 51.48), 6 : ("Urgot", 48.81), 110 : ("Varus", 46.22), 67 : ("Vayne", 54.34), 45 : ("Veigar", 55.74), 161 : ("Velkoz", 46.98), 254 : ("Vi", 54.12), 112 : ("Viktor", 51.93), 8 : ("Vladimir", 47.38), 106 : ("Volibear", 50.92), 19 : ("Warwick", 48.26), 498 : ("Xayah", 50.55), 101 : ("Xerath", 50.63), 5 : ("XinZhao", 49.71), 157 : ("Yasuo", 47.79), 777 : ("Yone", 49.74), 83 : ("Yorick", 47.69), 350 : ("Yuumi", 54.96), 154 : ("Zac", 49.89), 238 : ("Zed", 50.77), 115 : ("Ziggs", 39.52), 26 : ("Zilean", 51.02), 142 : ("Zoe", 48.62), 143 : ("Zyra", 48.91)}
winrate_map = {}
champion_names = []
for i in winrate_list :
  winrate_map[winrate_list[i][0]] = winrate_list[i][1]
  champion_names.append(winrate_list[i][0])

def load_model() :
  return pickle.load(open(root + '/ml_model_identity.pickle','rb'))

def get_winrate(names) :
  pass

def prediction(model, diffLevel, diffKill, diffTower, diffDragon, winrate) :
  data_t = torch.tensor([float(diffLevel), float(diffKill), float(diffTower), float(diffDragon), float(winrate)])
  data_t = data_t.unsqueeze(0)
  y_pred = model.predict(data_t)
  y_pred_prob = model.predict_log_proba(data_t)
  return y_pred, y_pred_prob

In [44]:
model = load_model()

level = input("Type difference of level : ")
kill = input("Type difference of kill : ")
tower = input("Type difference of tower : ")
dragon = input("Type difference of dragon : ")
winrates = 0
count = 0

while (count < 10) :
  teams = ['your ', 'opposite ']
  pos = ['top lane. ','jungle lane. ','mid lane.','AD carry. ', 'support. ']
  name = input("Type champion name of " + teams[count//5] + pos[count%5] +"If you don't know champion's English name, type 'help'. : ")
  if name in winrate_map :
    count += 1
    if (count//5 == 0) :
      winrates += winrate_map[name]
    else :
      winrates -= winrate_map[name]
  elif name == 'help' :
    print(champion_names)
  else :
    print("Wrong name. If you don't know champion's English name, type 'help'")

print("Calculating...")
pred, pred_prob = prediction(model, level, kill, tower, dragon, winrates)

if pred_prob[0, 0] >= pred_prob[0,1] :
  win_prob = np.exp(pred_prob[0,0]) / (np.exp(pred_prob[0,0]) + np.exp(pred_prob[0,1]))
else :
  win_prob = np.exp(pred_prob[0,1]) / (np.exp(pred_prob[0,0]) + np.exp(pred_prob[0,1]))

if pred[0] == 0 :
  print("15GG GoGo : you are likely to lose this game")
  print("Probability you lose this game is : %.1f %%"%(win_prob*100))
elif pred[0] == 1 :
  print("15GG? NoNo : you are likely to win this game")
  print("Probability you win this game is : %.1f %%"%(win_prob*100))

Type difference of level : 4
Type difference of kill : 7
Type difference of tower : 1
Type difference of dragon : 1
Type champion name of your top lane. If you don't know champion's English name, type 'help'. : help
['Aatrox', 'Ahri', 'Akali', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Blitzcrank', 'Brand', 'Braum', 'Caitlyn', 'Camile', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'M